In [118]:
import numpy as np
import json

In [112]:
GeneralCases = {}
eta = 0.1
pel = 10**(-7)
typeMap = {
    "id" : {"isId" : True, "isSymbol": False, "type" : str,},
    "int" : {"isId" : False, "isSymbol": False, "type" : int, "low" : None, "high" : None},
    "bool" : {"isId" : False, "isSymbol": False, "type" : json.loads, "low" : 0, "high" : 1},
    "symbol" : {"isId" : False, "isSymbol": True, "type" : str, "values" : []},
    }

In [113]:
class HOLO_Case:
    num_cases = 0
    cases = []

    @staticmethod
    def DefineAttributes(attributeMap, solution):
        HOLO_Case.attributes = attributeMap
        HOLO_Case.attributeValues = {k : None for k in attributeMap if k != solution}
        HOLO_Case.solution = solution

    def __init__(self, attributeValues):
        self.attributeWeights = {k : 0 for k in HOLO_Case.attributes if k != HOLO_Case.solution and not HOLO_Case.attributes[k]["isId"] and not HOLO_Case.attributes[k]["isSymbol"]}
        self.attributeValues = {k : attributeValues[k] for k in HOLO_Case.attributes if k != HOLO_Case.solution}
        self.solutionValue = attributeValues[HOLO_Case.solution]

    def SetGC(self, key):
        self.GC = key
        self.holoWeight = 0

    def SOLODistance(self, newCase):
        return sum([self.attributeWeights[k] * abs(self.attributeValues[k] - newCase.attributeValues[k]) for k in self.attributes if not HOLO_Case.attributes[k]["isId"] and not HOLO_Case.attributes[k]["isSymbol"] and k != HOLO_Case.solution])

    def Predict(self, newCase):
        return self.solutionValue + self.SOLODistance(newCase)

    def UpdateSOLO(self, newCase):
        predictedDiff = self.Predict(newCase) - newCase.solutionValue
        diffMap = {k : newCase.attributeValues[k] - self.attributeValues[k] for k in self.attributes if not HOLO_Case.attributes[k]["isId"] and not HOLO_Case.attributes[k]["isSymbol"] and k != HOLO_Case.solution}
        normSquare = np.linalg.norm(list(diffMap.values())) ** 2
        if normSquare != 0:
            self.attributeWeights = {k : self.attributeWeights[k] - diffMap[k] * (predictedDiff / normSquare) for k in self.attributes if not HOLO_Case.attributes[k]["isId"] and not HOLO_Case.attributes[k]["isSymbol"] and k != HOLO_Case.solution}

    def UpdateHOLO(self, newCase, predicted):
        diffCase = self.Predict(newCase) - newCase.solutionValue
        diffGC = predicted - newCase.solutionValue
        if abs(diffCase) <= abs(diffGC):
            self.holoWeight += 1
        else:
            self.holoWeight -= 1
        if self.holoWeight <= -5:
            del GeneralCases[self.GC].caseList[hash(self)]

In [114]:
class HOLO_GC(HOLO_Case):

    def __init__(self, newCase):
        self.caseList = {}
        self.holoLinks = {}
        
        self.solutionValue = newCase.solutionValue
        self.attributeValues = newCase.attributeValues
        self.attributeWeights = {k : 0 for k in newCase.attributeWeights}
        GeneralCases.setdefault(hash(self), self)

    def HOLODistance(self, gc, newCase):
        return sum([self.holoLinks[hash(gc)][k] * abs(gc.attributeValues[k] - newCase.attributeValues[k]) for k in self.attributes if not HOLO_Case.attributes[k]["isId"] and not HOLO_Case.attributes[k]["isSymbol"] and k != HOLO_Case.solution])

    def UpdateSOLO(self, newCase):
        solves = self.Solves(newCase)
        diff = {k : HOLO_Case.attributes[k]["high"] - HOLO_Case.attributes[k]["low"] - int(not(solves ^ self.attributeValues[k] == newCase.attributeValues[k])) for k in self.attributes if not HOLO_Case.attributes[k]["isId"] and not HOLO_Case.attributes[k]["isSymbol"] and k != HOLO_Case.solution}
        demon = sum(diff.values())
        if demon != 0:
          self.attributeWeights = {k : self.attributeWeights[k] + eta * diff[k] / demon for k in self.attributes if not HOLO_Case.attributes[k]["isId"] and not HOLO_Case.attributes[k]["isSymbol"] and k != HOLO_Case.solution}
          demon = sum(self.attributeWeights.values())
          self.attributeWeights = {k : self.attributeWeights[k] / demon for k in self.attributes if not HOLO_Case.attributes[k]["isId"] and not HOLO_Case.attributes[k]["isSymbol"] and k != HOLO_Case.solution}
    
    def UpdateHOLO(self, gc, newCase):
        if hash(gc) not in self.holoLinks:
            self.holoLinks.setdefault(hash(gc), {k : gc.attributeValues[k] - newCase.attributeValues[k] for k in self.attributes if not HOLO_Case.attributes[k]["isId"] and not HOLO_Case.attributes[k]["isSymbol"] and k != HOLO_Case.solution})
        else:
            solves = gc.Solves(newCase)
            diff = {k : HOLO_Case.attributes[k]["high"] - HOLO_Case.attributes[k]["low"] - int(not(solves ^ gc.attributeValues[k] == newCase.attributeValues[k])) for k in self.attributes if not HOLO_Case.attributes[k]["isId"] and not HOLO_Case.attributes[k]["isSymbol"] and k != HOLO_Case.solution}
            demon = sum(diff.values())
            if demon != 0:
              self.holoLinks[hash(gc)] = {k : self.holoLinks[hash(gc)][k] + eta * diff[k] / demon for k in self.attributes if not HOLO_Case.attributes[k]["isId"] and not HOLO_Case.attributes[k]["isSymbol"] and k != HOLO_Case.solution}
              demon = sum(self.holoLinks[hash(gc)].values())
              self.holoLinks[hash(gc)] = {k : self.holoLinks[hash(gc)][k] / demon for k in self.attributes if not HOLO_Case.attributes[k]["isId"] and not HOLO_Case.attributes[k]["isSymbol"] and k != HOLO_Case.solution}

    def AddCase(self, newCase):
        predicted = self.Predict(newCase)
        if abs(predicted - newCase.solutionValue) <= pel:
            for case in self.caseList.values():
                case.UpdateSOLO(newCase)
                case.UpdateHOLO(newCase, predicted)
            self.UpdateSOLO(newCase)
            for gc in self.holoLinks:
                self.UpdateHOLO(GeneralCases[gc], newCase)
            newCase.SetGC(hash(self))
            self.caseList.setdefault(hash(newCase), newCase)
        else:
            self.UpdateSOLO(newCase)
            transfers = {}
            for k in self.holoLinks:
                predicted = GeneralCases[k].Predict(newCase)
                if abs(predicted - newCase.solutionValue) <= pel:
                    transfers.setdefault(k, self.holoLinks[k])
            if transfers == {}:
                newGC = HOLO_GC(newCase)
                newGC.AddCase(newCase)
                GeneralCases.setdefault(hash(newGC), newGC)
                self.UpdateHOLO(newGC, newCase)                
                newGC.UpdateHOLO(self, newCase)
                # Initialize holo weights
            else:
                for gc in self.holoLinks:
                    self.UpdateHOLO(GeneralCases[gc], newCase)          
                    GeneralCases[gc].UpdateHOLO(self, newCase)
                    # Update holo links
    def Solve(self, newCase):
        if not self.Solves(newCase):
            closest = min([[self.HOLODistance(GeneralCases[case], newCase), case] for case in self.holoLinks])
            if closest < pel:
                return GeneralCases[closest].Solve(newCase)
        else:
            return min([[abs(case.Predict(newCase) - newCase.solutionValue), case.Predict(newCase)] for case in self.cases], key = lambda x : x[0])[1]

    def Solves(self, newCase):
        return self.SOLODistance(newCase) <= pel

In [115]:
class HOLO_System:
    def AddCase(self, newCase):
        if GeneralCases != {}:
            closestGC = min([[GeneralCases[k].SOLODistance(newCase), GeneralCases[k]] for k in GeneralCases], key = lambda x : x[0])[1]
            closestGC.AddCase(newCase)
        else:
            newGC = HOLO_GC(newCase)
            GeneralCases.setdefault(hash(newGC), newGC)
    def Solve(self, newCase):
        return min([[GeneralCases[k].SOLODistance(newCase), GeneralCases[k]] for k in GeneralCases])[1].Solve(newCase)

In [116]:
attributes = open("zoo.names", "r").read().split("\n")
attributes = [x.split(":") for x in attributes]
attributeMap = {}
for x in attributes:
    attributeMap.setdefault(x[0], typeMap[x[1]])
    if x[1] == "int":
        attributeMap[x[0]]["low"] = int(x[-2])
        attributeMap[x[0]]["high"] = int(x[-1])
    elif x[1] == "symbol":
        attributeMap[x[0]]["values"] = x[-1].split(",")
data = open("zoo.data", "r").read().split("\n")
data = [x.split(",") for x in data]

dataMap = [{k : attributeMap[k]["type"](x[i]) for i, k in enumerate(attributeMap)} for x in data]

HOLO_Case.DefineAttributes(attributeMap, "type")
system = HOLO_System()

# print(HOLO_Case.attributes)

for data in dataMap:
    newCase = HOLO_Case(data)
    system.AddCase(newCase)
print(len(GeneralCases.keys()))

for k in GeneralCases:
    print(len(GeneralCases[k].caseList))
    # for c in GeneralCases[k].caseList:
    #     print(GeneralCases[k].caseList[c].attributeValues)

39
2
6
11
7
2
4
3
4
3
2
3
1
1
3
4
4
3
2
2
2
1
6
2
3
2
2
1
1
1
1
1
1
1
1
1
1
1
1
1


In [117]:
    # def distance_gc(self, newCase):
    #     count = np.zeros([len(self.cases), d]) #len finds num_cases in gc
    #     solves = self.solve(newCase)[2]
    #     for case_idx in range(len(self.cases)):
    #         for attr_idx in range(d):
    #             if self.attributes[attr_idx] == newCase.attributes[attr_idx] and solves:
    #                 count[case_idx][attr_idx] += 1
    #             elif self.attributes[attr_idx] != newCase.attributes[attr_idx] and not solves:
    #                 count[case_idx][attr_idx] += 1
    #     return count


    # def update_adapt_gc(self, newCase, n):
    #     # g = np.argmin(holographic_case.retrieve(newCase, gc_list))
    #     count = self.distance_gc(newCase)
    #     dist = np.sum(count, axis=0, keepdims=True)/d #in gc class
    #     if holographic_case.num_cases == 1:
    #         self.adapt_weights_gc = np.zeros([d, 1])
    #     else:
    #         if self.solve(newCase)[2]: # predict return true if a gc is able to predict soln to newCase and false otherwise
    #             self.adapt_weights_gc += n*1/np.sum(np.max(dist)+np.min(dist)-dist)*(np.max(dist)+np.min(dist)-dist)
    #             self.adapt_weights_gc /= np.sum(self.adapt_weights_gc)
    #         else:
    #             self.adapt_weights_gc += n*1/np.sum(dist)*(dist)
    #             self.adapt_weights_gc /= np.sum(self.adapt_weights_gc)
    #     return self.adapt_weights_gc

    # @staticmethod
    # # returns dist to each gc
    # def retrieve(newCase, gc_list):
    #     ret = np.zeros([len(gc_list), d])
    #     for gc_idx, gc in enumerate(gc_list):
    #         for attr_idx in range(d):
    #             if gc.attributes[attr_idx] == newCase.attributes[attr_idx]:
    #                 ret[gc_idx][attr_idx] = 0
    #             elif gc.attributes[attr_idx] != newCase.attributes[attr_idx]:
    #                 ret[gc_idx][attr_idx] = gc.adapt_weights_gc[attr_idx]
    #     return np.sum(ret, axis=1, keepdims=True)


    # def solve(newCase):
    #     pass
    #     # return closest gc_i using retrieve func & f(x tilda) & check if d(x tilda) == groundtruth
    #     # returns a [gc_i index, f(x tilda) value, True/False]
